<a href="https://colab.research.google.com/github/aekanun2020/2022-PUB_COC-Data-Science-for-Tourism/blob/main/WORKSHOP_Spark_Missing_Exploration_with_MCAR_MNAR_MAR_Structural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
   .appName("Neural Network Model") \
   .config("spark.executor.memory", "3gb") \
   .getOrCreate()
   
sc = spark.sparkContext

sc

<SparkContext master=local[*] appName=pyspark-shell>

In [ ]:
! wget https://storage.googleapis.com/5-7sep2022/data/2008.csv

--2022-09-24 13:50:53--  https://storage.googleapis.com/5-7sep2022/data/2008.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.23.128, 74.125.203.128, 74.125.204.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.23.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689413344 (657M) [text/csv]
Saving to: ‘2008.csv’

2008.csv            100%[===================>] 657.48M   123MB/s    in 5.6s    

2022-09-24 13:50:59 (117 MB/s) - ‘2008.csv’ saved [689413344/689413344]



In [ ]:
raw_df = spark.read.option('header','true').csv('./2008.csv')

In [ ]:
raw_df.count()

7009728

In [ ]:
raw_df.head(3)

[Row(Year='2008', Month='1', DayofMonth='3', DayOfWeek='4', DepTime='2003', CRSDepTime='1955', ArrTime='2211', CRSArrTime='2225', UniqueCarrier='WN', FlightNum='335', TailNum='N712SW', ActualElapsedTime='128', CRSElapsedTime='150', AirTime='116', ArrDelay='-14', DepDelay='8', Origin='IAD', Dest='TPA', Distance='810', TaxiIn='4', TaxiOut='8', Cancelled='0', CancellationCode=None, Diverted='0', CarrierDelay='NA', WeatherDelay='NA', NASDelay='NA', SecurityDelay='NA', LateAircraftDelay='NA'),
 Row(Year='2008', Month='1', DayofMonth='3', DayOfWeek='4', DepTime='754', CRSDepTime='735', ArrTime='1002', CRSArrTime='1000', UniqueCarrier='WN', FlightNum='3231', TailNum='N772SW', ActualElapsedTime='128', CRSElapsedTime='145', AirTime='113', ArrDelay='2', DepDelay='19', Origin='IAD', Dest='TPA', Distance='810', TaxiIn='5', TaxiOut='10', Cancelled='0', CancellationCode=None, Diverted='0', CarrierDelay='NA', WeatherDelay='NA', NASDelay='NA', SecurityDelay='NA', LateAircraftDelay='NA'),
 Row(Year='20

In [ ]:
raw_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
Year,7009728,2008.0,1.6928719673098015E-13,2008,2008
Month,7009728,6.375129676928976,3.4067366032507986,1,9
DayofMonth,7009728,15.72801483880687,8.797067919493633,1,9
DayOfWeek,7009728,3.9241815088973495,1.9882589459851212,1,7
DepTime,7009728,1333.8300461105448,478.06889486629836,1,NA
CRSDepTime,7009728,1326.0856632382884,464.2509106889191,0,959
ArrTime,7009728,1481.258226684178,505.22512933801556,1,NA
CRSArrTime,7009728,1494.8011536253618,482.67282151078666,0,959
UniqueCarrier,7009728,None,None,9E,YV


In [ ]:
from pyspark.sql import functions as sparkf

In [ ]:
##พบว่ามี Null เพียงแค่ 2 Columns เท่านั้น
raw_df.select([sparkf.count(sparkf.when(sparkf.isnull(c), c)).alias(c) for c in raw_df.columns])\
.toPandas().transpose()

,0
Year,0
Month,0
DayofMonth,0
DayOfWeek,0
DepTime,0
CRSDepTime,0
ArrTime,0
CRSArrTime,0
UniqueCarrier,0
FlightNum,0


In [ ]:
##พบว่ามี NA อยู่หลาย Columns 
raw_df.select([sparkf.count(sparkf.when(sparkf.col(c)=='NA', c)).alias(c) for c in raw_df.columns])\
.toPandas().transpose()

,0
Year,0
Month,0
DayofMonth,0
DayOfWeek,0
DepTime,136246
CRSDepTime,0
ArrTime,151649
CRSArrTime,0
UniqueCarrier,0
FlightNum,0


In [ ]:
##พบว่าไม่มี empty values เลย
raw_df.select([sparkf.count(sparkf.when(sparkf.col(c)=='', c)).alias(c) for c in raw_df.columns])\
.toPandas().transpose()

,0
Year,0
Month,0
DayofMonth,0
DayOfWeek,0
DepTime,0
CRSDepTime,0
ArrTime,0
CRSArrTime,0
UniqueCarrier,0
FlightNum,0


In [ ]:
from functools import reduce

raw_df.filter(reduce(lambda a1, a2: a1 | a2,\
                     (sparkf.col(c).isNull() \
                      for c in raw_df.columns)))\
.sample(0.0001)\
.toPandas().transpose()

,0,1,2,3,4,5,6,7,8,9,...,672,673,674,675,676,677,678,679,680,681
Year,2008,2008,2008,2008,2008,2008,2008,2008,2008,2008,...,2008,2008,2008,2008,2008,2008,2008,2008,2008,2008
Month,1,1,1,1,1,1,1,1,1,1,...,12,12,12,12,12,12,12,12,12,12
DayofMonth,5,6,7,9,11,11,14,27,29,30,...,19,1,7,5,17,1,12,31,29,11
DayOfWeek,6,7,1,3,5,5,1,7,2,3,...,5,1,7,5,3,1,5,3,1,4
DepTime,930,2135,858,1646,1911,2028,659,1628,1522,1352,...,645,736,1902,1825,1643,2144,1436,1849,2214,954
CRSDepTime,920,2120,900,1650,1910,2030,700,1610,1525,1350,...,640,740,1855,1825,1645,2145,1400,1815,2205,955
ArrTime,1130,2230,1015,1823,1915,2154,955,1712,2240,1454,...,1108,1629,2133,2216,1846,17,1527,2149,610,1212
CRSArrTime,1055,2210,1020,1820,1915,2200,950,1700,2255,1455,...,1000,1530,2200,2245,1845,40,1448,2102,617,1146
UniqueCarrier,WN,WN,WN,WN,WN,WN,WN,WN,WN,WN,...,AA,AA,AA,AA,AA,AA,AS,AS,B6,DL
FlightNum,905,1493,1486,1679,160,1172,1097,3419,446,1620,...,201,202,397,762,1320,1640,64,129,90,728


In [ ]:
from functools import reduce

raw_df.filter(reduce(lambda a1, a2: a1 | a2,\
                     (sparkf.col(c).isNull() \
                      for c in raw_df.columns)))\
.select([n for n in raw_df.columns if ('CancellationCode' in n)|('TailNum' in n)\
        | ('Time' in n)]).show(1000)

+-------+----------+-------+----------+-------+-----------------+--------------+-------+----------------+
|DepTime|CRSDepTime|ArrTime|CRSArrTime|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|CancellationCode|
+-------+----------+-------+----------+-------+-----------------+--------------+-------+----------------+
|   2003|      1955|   2211|      2225| N712SW|              128|           150|    116|            null|
|    754|       735|   1002|      1000| N772SW|              128|           145|    113|            null|
|    628|       620|    804|       750| N428WN|               96|            90|     76|            null|
|    926|       930|   1054|      1100| N612SW|               88|            90|     78|            null|
|   1829|      1755|   1959|      1925| N464WN|               90|            90|     77|            null|
|   1940|      1915|   2121|      2110| N726SW|              101|           115|     87|            null|
|   1937|      1830|   2037|      1940| N763SW

In [ ]:
raw_df.select([sparkf.count(sparkf.when(sparkf.isnull(c), c)).alias(c) for c in raw_df.columns])\
.toPandas().transpose()

,0
Year,0
Month,0
DayofMonth,0
DayOfWeek,0
DepTime,0
CRSDepTime,0
ArrTime,0
CRSArrTime,0
UniqueCarrier,0
FlightNum,0


In [ ]:
raw_df.select(sparkf.when(sparkf.isnull('TailNum'), 'TailNum')).distinct().show()

+--------------------------------------------+
|CASE WHEN (TailNum IS NULL) THEN TailNum END|
+--------------------------------------------+
|                                        null|
|                                     TailNum|
+--------------------------------------------+



In [ ]:
condCol = sparkf.when(sparkf.isnull('TailNum'), 'TailNum')

In [ ]:
raw_df.select(sparkf.count(condCol)).show()

+---------------------------------------------------+
|count(CASE WHEN (TailNum IS NULL) THEN TailNum END)|
+---------------------------------------------------+
|                                              83365|
+---------------------------------------------------+



In [ ]:
raw_df.select(sparkf.count(condCol).alias('TailNum')).show()

+-------+
|TailNum|
+-------+
|  83365|
+-------+



In [ ]:
from functools import reduce

raw_df.filter(reduce(lambda a1, a2: a1 | a2,\
                     (sparkf.col(c).isNull() \
                      for c in raw_df.columns)))\
.select([n for n in raw_df.columns if 'Delay' in n]).show(1000)

+--------+--------+------------+------------+--------+-------------+-----------------+
|ArrDelay|DepDelay|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+--------+--------+------------+------------+--------+-------------+-----------------+
|     -14|       8|          NA|          NA|      NA|           NA|               NA|
|       2|      19|          NA|          NA|      NA|           NA|               NA|
|      14|       8|          NA|          NA|      NA|           NA|               NA|
|      -6|      -4|          NA|          NA|      NA|           NA|               NA|
|      34|      34|           2|           0|       0|            0|               32|
|      11|      25|          NA|          NA|      NA|           NA|               NA|
|      57|      67|          10|           0|       0|            0|               47|
|     -18|      -1|          NA|          NA|      NA|           NA|               NA|
|       2|       2|          NA|          N

In [ ]:
from functools import reduce

raw_df.filter(reduce(lambda a1, a2: a1 | a2,\
                     (sparkf.col(c).isNull() \
                      for c in raw_df.columns)))\
.filter((sparkf.col('DepTime') == 'NA')\
        &(sparkf.col('ArrTime') == 'NA'))\
.sample(0.001).toPandas().transpose()

,0,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,77
Year,2008,2008,2008,2008,2008,2008,2008,2008,2008,2008,...,2008,2008,2008,2008,2008,2008,2008,2008,2008,2008
Month,1,1,1,1,1,1,1,1,2,2,...,8,9,9,9,9,10,11,12,12,12
DayofMonth,22,17,4,3,24,29,7,31,4,12,...,4,10,14,15,14,23,30,22,9,14
DayOfWeek,2,4,5,4,4,2,1,4,1,2,...,1,3,7,1,7,4,7,1,2,7
DepTime,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
CRSDepTime,1540,1600,2220,1720,1350,1515,1030,1815,835,1235,...,2040,1145,1240,920,920,1725,2010,605,800,1620
ArrTime,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
CRSArrTime,1800,1945,2355,1810,2045,1659,1120,1945,1130,1510,...,2355,1305,1432,1145,1035,1910,2238,750,913,1720
UniqueCarrier,XE,XE,UA,WN,HA,MQ,MQ,AA,WN,WN,...,AA,WN,XE,OH,AA,9E,OH,WN,US,9E
FlightNum,2260,1226,647,373,12,3999,4234,429,1519,1247,...,1358,851,2509,6837,567,5657,6937,3019,2162,2838


In [ ]:
## ได้ข้อสรุปว่าทุกครั้งที่ Cancelled เป็น 1 จะส่งผลให้ DepTime = 'NA' และ ArrTime = 'NA'
from functools import reduce

raw_df.filter(reduce(lambda a1, a2: a1 | a2,\
                     (sparkf.col(c).isNull() \
                      for c in raw_df.columns)))\
.filter((sparkf.col('DepTime') == 'NA')\
        &(sparkf.col('ArrTime') == 'NA'))\
.groupBy('Cancelled').count().orderBy('count',ascending=False).show()

+---------+-----+
|Cancelled|count|
+---------+-----+
|        1|83355|
+---------+-----+



In [ ]:
## พิสูจน์
raw_df.groupBy('Cancelled').count().show()

+---------+-------+
|Cancelled|  count|
+---------+-------+
|        0|6872294|
|        1| 137434|
+---------+-------+

